In [ ]:
%load_ext autoreload
%autoreload 2
import os
import random

import hydra
import matplotlib.pyplot as plt
import numpy as np
import torch
from dataloader import ARMA
from dataloader import ElectricDevices
from hydra.experimental import compose
from hydra.experimental import initialize_config_dir
from interpretability import get_hist
from interpretability import global_interpretability
from interpretability import shapley_bottleneck_vs_class
from interpretability import shapley_bottleneck_vs_output
from interpretability import shapley_input_vs_bottleneck
from interpretability import shapley_input_vs_class
from interpretability import shapley_input_vs_output
from models.CAE import CAE
from train import Trainer
from utils import accuracy
from utils import baseline
from utils import data_bottleneck_exploration
from utils import data_input_exploration
from utils import get_predictions
from utils import observation_reconstruction
from utils import reconstruction

torch.manual_seed(4444)
np.random.seed(4444)
random.seed(4444)

plt.rcParams.update({'font.size': 14})

In [ ]:
TRAIN = False # Use a pretained model or train one from 0.
TYPE = 3 # 1: Random observation per class. 2: Default ARMA. 3: Default ElectricDevices.

# Model

In [ ]:
with initialize_config_dir(config_dir=os.path.abspath("configs")):
    cfg = compose(config_name="config")

dl = ElectricDevices()
#dl = ARMA(5)

data_train, data_valid, data_test = dl()
X_train, y_train = data_train[:, :, :-1], data_train[:, :, -1]
X_valid, y_valid = data_valid[:, :, :-1], data_valid[:, :, -1]
X_test, y_test = data_test[:, :, :-1], data_test[:, :, -1]

In [ ]:
model = CAE(cfg.model)

In [ ]:
if TRAIN:
    trainer = Trainer(cfg.train)
    train_losses, valid_losses = trainer.fit(model, data_train, data_valid)

    plt.title("Cost")
    plt.plot(train_losses, label="train")
    plt.plot(valid_losses, label="validation")
    plt.legend()

else:
    model.load_state_dict(torch.load("../weights/mod.pth"))

# Uncomment to save the model
#torch.save(model.state_dict(), "../weights/mod_checkpoint.pth")

Predict testing data,

In [ ]:
X_testp, y_testp, bn = get_predictions(model, X_test)

# Evaluate

## Reconstruction

In [ ]:
reconstruction(X_test, X_testp, y_test)

### Reconstruction of individual observations

Select the predefined observations per problem (TYPE 2 or 3) or select random ones.

Execute this cell and the next one several times with TYPE 1 to see several observations per class

In [ ]:
if TYPE == 1:
    # Random selection, one per class
    SELECTED = [np.random.choice([i for i, x in enumerate(y_test) if int(x) == j]) for j in range(cfg.model.num_classes)]
elif TYPE == 2:
    # ARMA
    SELECTED = [477, 250, 865, 113]
else:
    # ELECTRIC DEVICES
    SELECTED = [3279, 1156, 7419, 5046, 3323, 6485, 5497]

In [ ]:
observation_reconstruction(SELECTED, X_test, X_testp)
print("Real class:", y_test[SELECTED].flatten().detach().numpy().astype(int))
print("Predicted class:", y_testp[SELECTED].flatten())

## Classification

In [ ]:
accuracy(y_test, y_testp)

In [ ]:
baseline(data_train, data_valid, data_test)

# Data Exploration

Distribution of the input,

In [ ]:
data_input_exploration(X_train)

Distribution of the bottleneck neurons,

In [ ]:
data_bottleneck_exploration(model, X_train)

# Interpretability

## Global Interpretability (Importance Matrix from the weights)

In [ ]:
global_interpretability(model)

## Local Interpretability (Shapley Value)

Obtain data distribution from which perform the sampling.

We are going to compute the importance of input positions or bottleneck neurons.

In [ ]:
hist_input = [get_hist(X_train[:, 0, i]) for i in range(cfg.model.length)]
aux = model.encoder(X_train, False).detach().numpy()
hist_bn = [get_hist(aux[:, i]) for i in range(cfg.model.bottleneck_nn)]

Shapley Values for different parts of the autoencoder. For example, important of the input positions to reconstruct an output position.

In [ ]:
shapley_input_vs_output(model, SELECTED, X_test, hist_input)

In [ ]:
shapley_bottleneck_vs_output(model, SELECTED, X_test, hist_bn)

In [ ]:
shapley_input_vs_bottleneck(model, SELECTED, X_test, hist_input)

In [ ]:
shapley_bottleneck_vs_class(model, SELECTED, X_test, hist_bn)

In [ ]:
shapley_input_vs_class(model, SELECTED, X_test, hist_input)